## 06. 딥러닝의 미래 GAN

### 대립하는(adversarial)하는 두 신경망을 경쟁시켜가며 결과물 생성 방법을 학습

<font size=4 color='blue'> 
실제이미지를 구분자(Discriminator)에게 이 이미지가 진짜임을 판단하게 하고, 생성자(Generator)를 통해 노이즈로부터 임의의 이미지를 만들고 이것을 다시 같은 구분자를 통해 진짜 이미지인지를 판단하게 한다.
</font>

### 응용 예 <br>

<font size=4 color='blue'> 
(가) 고흐 풍 그림으로 그려주기 <br>
(나) 선으로 그려진 만화를 자동으로 채색 <br>
(다) 모자이크를 없애주기 <br>
(라) GAN 기법을 이용한 자연어 문장 생성
</font>

## 예제 - MNIST 손글씨를 무작위로 생성하는 예제 만들기

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


### 하이퍼 파리미터 설정

In [0]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28
n_noise = 128    # 생성자의 입력값 - 사용할 노이즈의 크기 (노이즈에서 손글씨 이미지를 무작위 생성할 계획) 
n_class = 10

In [0]:
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])  # 노이즈 입력 

### 생성자 신경망에 사용할 변수들을 설정
 * 첫번째 가중치와 편향(은닉층)
 * 두번째 가중치와 편향(출력층) - 출력층에 사용해야 하므로 실제 이미지의 크기와 같아야 함. 28 X 28 = 784

In [0]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_input]))

### 구분자 신경망에 사용할 변수들을 설정
 * 은닉층과 동일하게 구성
 * 구분자는 진짜와 얼마나 가까운가를 판단하는 값임.(0~1사이의 값)- 하나의 값
 * 구분자 신경망(진짜를 판별, 생성자에서 생성한 이미지 판별)은 같은 변수 사용해야 함.

In [0]:
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

### 생성자와 구분자의 신경망을 구성

In [0]:
def generator(noise_z):
    hidden = tf.nn.relu( tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    
    return output

### 구분자 신경망

In [0]:
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

### 무작위한 노이즈를 만들어 주는 간단한 유틸리티 함수

In [0]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

## 노이즈 Z를 이용하여 가짜 이미지를 만들 생성자 G를 만들고
## 이 G가 만든 가짜 이미지와 진짜 이미지 X를 각각 구분자에 넣어 
## 입력한 이미지가 진짜인지를 판별.

In [0]:
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

ValueError: Dimensions must be equal, but are 256 and 784 for 'add_3' (op: 'Add') with input shapes: [?,256], [784].

### 데이터 업데이트